<a href="https://colab.research.google.com/github/emnaNaitIbourk/PRODGY_GA_01/blob/main/GA_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q
!pip install torch -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00


In [3]:
#Imports
from transformers import GPT2LMHeadModel,GPT2Tokenizer,Trainer,TrainingArguments,TextDataset,DataCollatorForLanguageModeling
import torch
#Loading the tokenizer and model
print("Loading tokenizer and model")
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
model=GPT2LMHeadModel.from_pretrained("gpt2")
#making sure that the model works  correctly
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    #preparing dataset
print("Preparing dataset")
dataset=TextDataset(
    tokenizer=tokenizer,
    file_path="mydataset.txt",
    block_size=128,


)
#Organizing data into batches for efficient training
dataCollator=DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,

 )
#setting training rules
import os
os.environ["WANDB_DISABLED"] = "true"

training_args=TrainingArguments(
    output_dir="./finetuned_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,

)
#Finetuning
print("Starting fine-tuning...")
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=dataCollator,
    train_dataset=dataset,


)
#Starting the training process
trainer.train()
#Saving the finetuned model and its tokenizer to the new folder
print("Finetuning is complete.Saving model...")
model.save_pretrained("./finetuned_gpt2")
tokenizer.save_pretrained("./finetuned_gpt2")
print("Model and Tokenizer saved  to  './finetuned_gpt2' .")

Loading tokenizer and model


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Preparing dataset
Starting fine-tuning...


Step,Training Loss


Finetuning is complete.Saving model...
Model and Tokenizer saved  to  './finetuned_gpt2' .


In [5]:
!pip install streamlit -q
!pip install pyngrok -q


In [9]:
%%writefile app.py
 #Imports for Streamlit and Hugging Face.
import streamlit as st
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
import os

# This sets the title for your app's web page.
st.title("My Custom Text Generator")

# This is the folder where your fine-tuned model is saved.
model_path = "./finetuned_gpt2"

# This part checks if the model exists before trying to use it.
if not os.path.exists(model_path):
    st.error(f"Model not found at {model_path}. Please run the training script first.")
else:
    # `@st.cache_resource` tells Streamlit to load the model only ONCE, which makes the app fast.
    @st.cache_resource
    def get_generator():
        # `pipeline` is a simple tool that loads the model and tokenizer together for easy text generation.
        return pipeline('text-generation', model=model_path, tokenizer=model_path)

    generator = get_generator()

    # `st.text_area` creates a box where a user can type in their starting sentence.
    prompt = st.text_area("Enter your prompt:", "Once upon a time...")

    # These create sliders on the side of the page to adjust how the text is generated.
    max_length = st.sidebar.slider("Max text length", 50, 500, 150)
    temperature = st.sidebar.slider("Creativity (higher = more random)", 0.1, 1.0, 0.7)

    # `st.button` creates a button that, when clicked, runs the code inside the `if` statement.
    if st.button("Generate Text"):
        if prompt:
            st.write("Generating...")
            # `generator(...)` is where the magic happens! It uses your fine-tuned model to continue the text.
            generated_text = generator(
                prompt,
                max_length=max_length,
                temperature=temperature
            )

            # These lines display the generated text in a clean format on the web page.
            st.subheader("Generated Text:")
            st.markdown(generated_text[0]['generated_text'])
        else:
            st.warning("Please enter a prompt.")

Overwriting app.py


In [7]:
!streamlit run app.py &>/dev/null &


In [8]:
from pyngrok import ngrok

# Set your ngrok authentication token
ngrok.set_auth_token("31KEfRtpjNt5NBwI442Fu4w57AO_5bViZBm4c6DXSvGTiud3u")

# Open a tunnel to port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print("Your public URL:", public_url)


Your public URL: NgrokTunnel: "https://5a55636f9307.ngrok-free.app" -> "http://localhost:8501"
